### Imports and Configurations

In [17]:
import cv2
import numpy as np

### Load Model

In [20]:
# Download files
# https://pjreddie.com/media/files/yolov3.weights
# https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
# https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

In [19]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

### Load Image

In [9]:
image = cv2.imread("image_for_detection.jpg")
height, width, channels = image.shape

blob = cv2.dnn.blobFromImage(image, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []

### Processing Detections

In [10]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5: 
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(classes), 3))

### Show results

In [11]:
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y + 30), font, 3, color, 3)

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()